In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import sys
from datetime import datetime

today = datetime.today().strftime("%Y-%m-%d")

MISSION     = 'Kepler'
PROJECT_DIR = '/Users/research/projects/kepler-ecc-rp/'
RUN_ID      = 'ecc-all-LC-omc-refit-20240924'

_proj = '/data/user/gjgilbert/projects/alderaan/'
_cat  = 'kepler_dr25_gaia_dr2_crossmatch.csv'
_data = '/data/user/gjgilbert/data/MAST_downloads/'

In [2]:
MULTIPLICITY = (1,99)
PER_LIM      = (1,300)
RAD_TYPE     = 'rp'
RAD_LIM      = (0.5,16)
MSTAR_LIM    = (0.,10.)
RSTAR_LIM    = (0.7,1.4)
FEH_LIM      = (-1.0,1.0)
TEFF_LIM     = (4700,6500)
AGE_LIM      = (0,14)

## Load DR25 data

In [3]:
DR25_CATALOG = os.path.join(PROJECT_DIR, 'Catalogs/kepler_dr25_gaia_dr2_crossmatch.csv')
catalog = pd.read_csv(DR25_CATALOG, index_col=0)

# hard-code period and radius limits
use  = (catalog.period > 1) * (catalog.period < 300)
use *= (catalog.rp > 0) * (catalog.rp < 16)

# clean up stellar sample
use *= catalog.logg > 4.0                                 # surface gravity as proxy for main sequence
use *= ((catalog.rcf - 1) < 0.05) + np.isnan(catalog.rcf) # radius correction factor (Furlan+ 2017)
use *= catalog.ruwe < 1.4                                 # Gaia RUWE

# slice subpopulation
use *= ((catalog.npl >= MULTIPLICITY[0]) &
        (catalog.npl <= MULTIPLICITY[1]) &
        (catalog.period >= PER_LIM[0]) &
        (catalog.period <= PER_LIM[1]) &
        (catalog.mstar >= MSTAR_LIM[0]) &
        (catalog.mstar <= MSTAR_LIM[1]) &
        (catalog.rstar >= RSTAR_LIM[0]) &
        (catalog.rstar <= RSTAR_LIM[1]) &
        (catalog.feh >= FEH_LIM[0]) &
        (catalog.feh <= FEH_LIM[1]) &
        (catalog.teff >= TEFF_LIM[0]) &
        (catalog.teff <= TEFF_LIM[1]) &
        (catalog.age >= AGE_LIM[0]) &
        (catalog.age <= AGE_LIM[1])
       )

# update catalog
catalog = catalog.loc[use].reset_index(drop=True)
targets = np.array(catalog.planet_name)

stars = np.sort(np.unique(catalog.koi_id))

In [5]:
stars = ['K02305',
 'K02363',
 'K02380',
 'K02507',
 'K02512',
 'K02555',
 'K02560',
 'K02594',
 'K02617',
 'K02660',
 'K02856',
 'K03017',
 'K03232',
 'K03396',
 'K03436',
 'K03472',
 'K03966',
 'K04037',
 'K04051',
 'K04251',
 'K04255',
 'K04411',
 'K04466',
 'K04613',
 'K04655',
 'K04710',
 'K04765',
 'K04776',
 'K04801',
 'K04815',
 'K05288',
 'K05632',
 'K05732',
 'K06179',
 'K06182',
 'K06209',
 'K06765',
 'K06931',
 'K07252',
 'K07359',
 'K07628',
 'K07635',
 'K07644',
 'K07679',
 'K07715',
 'K07749']

#### detrend_and_estimate_ttvs

In [6]:
file = '../bin/ALDERAAN_detrend_and_estimate_ttvs-{0}.tot'.format(today)

with open(file, 'w') as f:
    f.write('#!/bin/sh\n')
    
    for s in stars:
        theano_str  = "THEANO_FLAGS='base_compiledir=/data/user/gjgilbert/projects/alderaan/.theano/,compiledir_format={0}'".format(s)
        python_str  = "python bin/detrend_and_estimate_ttvs.py"
        args_str    = "--mission='{0}' --target='{1}' --project_dir={2} --catalog={3} --data_dir={4} --run_id={5}".format(MISSION, s, _proj, _cat, _data, RUN_ID)
        log_str     = "Logs/{0}/detrend_and_estimate_ttvs_{1}.log".format(RUN_ID, s)
        cleanup_str = "rm -r /data/user/gjgilbert/projects/alderaan/.theano/{0}".format(s)

        if MISSION == 'Simulated':
            args_str += " --run_id={0}".format(RUN_ID)
        
        f.write(theano_str + " " + python_str + " " + args_str + " &> " + log_str + " & wait ; " + cleanup_str + " &\n")     

#### analyze_autocorrelated_noise

In [7]:
file = PROJECT_DIR + 'bin/ALDERAAN_analyze_autocorrelated_noise-{0}.tot'.format(today)

with open(file, 'w') as f:
    f.write('#!/bin/sh\n')

    for s in stars:
        theano_str  = "THEANO_FLAGS='base_compiledir=/data/user/gjgilbert/projects/alderaan/.theano/,compiledir_format={0}'".format(s)
        python_str  = "python bin/analyze_autocorrelated_noise.py"
        args_str    = "--mission='{0}' --target='{1}' --project_dir={2} --catalog={3} --data_dir={4} --run_id={5}".format(MISSION, s, _proj, _cat, _data, RUN_ID)
        log_str     = "Logs/{0}/analyze_autocorrelated_noise_{1}.log".format(RUN_ID, s)
        cleanup_str = "rm -r /data/user/gjgilbert/projects/alderaan/.theano/{0}".format(s)
        
        if MISSION == 'Simulated':
            args_str += " --run_id={0}".format(RUN_ID)

        f.write(theano_str + " " + python_str + " " + args_str + " &> " + log_str + " & wait ; " + cleanup_str + " &\n") 

#### fit_transit_shape_simultaneous_nested_gp

In [8]:
file = PROJECT_DIR + 'bin/ALDERAAN_fit_transit_shape_simultaneous_nested-{0}.tot'.format(today)

with open(file, 'w') as f:
    f.write('#!/bin/sh\n')

    for s in stars:    
        # pull relevant quantities and establish GLOBAL variables
        use = np.array(catalog.koi_id) == s

        theano_str  = "THEANO_FLAGS='base_compiledir=/data/user/gjgilbert/projects/alderaan/.theano/,compiledir_format={0}'".format(s)
        python_str  = "python bin/fit_transit_shape_simultaneous_nested.py"
        args_str    = "--mission='{0}' --target='{1}' --project_dir={2} --catalog={3} --data_dir={4} --run_id={5}".format(MISSION, s, _proj, _cat, _data, RUN_ID)
        log_str     = "Logs/{0}/fit_transit_shape_simultaneous_nested_{1}.log".format(RUN_ID, s)
        cleanup_str = "rm -r /data/user/gjgilbert/projects/alderaan/.theano/{0}".format(s)
        
        if MISSION == 'Simulated':
            args_str += " --run_id={0}".format(RUN_ID)

        f.write(theano_str + " " + python_str + " " + args_str + " &> " + log_str + " & wait ; " + cleanup_str + " &\n") 